<a href="https://colab.research.google.com/github/lcylmhlcy/pytorch1.0_MostUse/blob/master/pytorch1_0_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! nvidia-smi

Thu Apr 25 12:15:31 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    31W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import collections
import os
import shutil
import tqdm

import numpy as np
import PIL.Image
import torch
import torchvision

In [5]:
print(torch.__version__)               # PyTorch version
print(torch.version.cuda)              # Corresponding CUDA version
print(torch.backends.cudnn.version())  # Corresponding cuDNN version
print(torch.cuda.get_device_name(0))   # GPU type

1.0.1.post2
10.0.130
7402
Tesla T4


In [7]:
# 判断是否有cuda支持
print(torch.cuda.is_available())

True


In [0]:
# 设置为 cuDNN benchmark 模式

# Benchmark 模式会提升计算速度，但是由于计算中有随机性，每次网络前馈结果略有差异。
torch.backends.cudnn.benchmark = True

# 如果想要避免这种结果波动，设置
torch.backends.cudnn.deterministic = True

# Note:
# 总的来说，大部分情况下，设置这个 benchmark 可以让内置的 cuDNN 的 auto-tuner 自动寻找最适合当前配置的高效算法，来达到优化运行效率的问题。
# 一般来讲，应该遵循以下准则：
# (1)如果网络的输入数据维度或类型上变化不大，设置  torch.backends.cudnn.benchmark = true  可以增加运行效率；
# (2)如果网络的输入数据在每次 iteration 都变化的话，会导致 cnDNN 每次都会去寻找一遍最优配置，这样反而会降低运行效率。

In [0]:
# 清除 GPU 存储

# 有时 Control-C 中止运行后 GPU 存储没有及时释放，需要手动清空。在 PyTorch 内部可以
torch.cuda.empty_cache()

# 或在命令行可以先使用 ps 找到程序的 PID，再使用 kill 结束该进程
ps aux | grep pythonkill -9 [pid]

# 或者直接重置没有被清空的 GPU
nvidia-smi --gpu-reset -i [gpu_id]

In [0]:
# 在训练开始时，参数的初始化是随机的，为了让每次的结果一致，我们需要设置随机种子。

# 固定GPU随机化种子
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)

# 为CPU设置随机种子
torch.manual_seed(args.seed)

In [0]:
# Tensor--------------------------------------------------------------------------------

In [12]:
a = torch.Tensor([[1,2,3],[4,5,6]])
print(a.type())
print(a.size())
print(a.dim())

torch.FloatTensor
torch.Size([2, 3])
2


In [0]:
# Set default tensor type. Float in PyTorch is much faster than double.
torch.set_default_tensor_type(torch.FloatTensor)

In [0]:
# Type convertions.
a = a.cuda() # convert Tensor a on CPU to Tensor a on GPU
a = a.cpu() # convert Tensor a on GPU to Tensor a on CPU

In [16]:
b = a.float() # convert a to FloatTensor
print(b.type())
c = a.long() # convert a to LongTensor
print(c.type())

torch.FloatTensor
torch.LongTensor


In [0]:
# torch.Tensor -> np.ndarray.
a = a.cuda()
a = a.cpu().numpy()  # convert to numpy on CPU

# np.ndarray -> torch.Tensor.
a1 = np.array([[1,2,3],[4,5,6]], dtype=np.int64)
b1 = torch.from_numpy(ndarray).float()
c1 = torch.from_numpy(ndarray.copy()).float()  # If ndarray has negative stride